##### Anisotropia a partir da Magnetização Remanente.
Este script varre a pasta dos dadosNormalizados localizando os valores de Magnetização por Magnetização de Saturação para localizar o valor de Magnetização quando o campo é zero para ambas as metades da curva (conjuntos de dados da "descida" e "subida") para cada ângulo, descobrindo assim o que chamamos de Magnetização Remanente (Mr)

In [ ]:
"""
CARREANDO BIBLIOTECAS NECESSÁRIAS
"""

import pandas as pd #manipulação de dados -ABQ
import numpy as np #manipulação de arrays e aplicações matemáticas -CLEIZA #type: ignore
import matplotlib.pyplot as plt #plot de figuras -CLEIZA #type: ignore
import os #manipulação de diretórios/arquivos -CLEIZA
from caminhosDiretorios import dirDadosDesteDispositivo as dirDados #variavel que guarda o caminho do diretório dos arquivos de medida -ABQ
from funcoesNomeDeArquivo import extrairPartesDoNome, extrair_angulo, nomeConcatenadoDaAmostra #de funcoesNomeDeArquivo.py, chamamos aqui funcoes que extraem partes do nome do arquivo -ABQ
from funcoesDeMatrizENormalizacao import metadeDoVetor, valorAbs #de funcoesDeMatrizENormalizacao.py, chamamos aqui funcoes que dividem o array em duas partes e encontram o valor absoluto de um array -ABQ


"""
CHAMANDO E RECONHECENDO DADOS
"""

#subdiretório onde estao localizados os dados normalizados a serem plotados -ABQ
subdiretorioAVarrer = os.path.join(dirDados+"dadosNormalizados/")

# Lista todos os arquivos na pasta -CLEIZA
arquivos = os.listdir(subdiretorioAVarrer)

# Filtra apenas os arquivos .txt -CLEIZA
arquivos_txt = [arquivo for arquivo in arquivos if arquivo.endswith('.txt')]

"""
OBTENDO VALORES DE M/Ms QUANDO O CAMPO É ZERO PARA CADA ÂNGULO
"""

#matriz que armazena os valores de M/Ms quando o campo é zero -ABQ
remanencias = []

# Obtendo os valores de M/Ms quando o campo é zero - tanto na "descida" quanto na "subida" (para cada ângulo) -CLEIZA
for arquivo_txt in arquivos_txt:
    
    nomeConcatenadoDaAmostraAlvo = nomeConcatenadoDaAmostra(arquivo_txt)

    # Extrai o ângulo do nome do arquivo -CLEIZA
    angulo = int(extrair_angulo(arquivo_txt))

    # Caminho completo do arquivo -CLEIZA
    caminho_arquivo = os.path.join(subdiretorioAVarrer, arquivo_txt)

    # Carrega os dados do arquivo -CLEIZA
    dados = pd.read_csv(caminho_arquivo, delimiter=" ", header=None, dtype=float)

    #divide o array em duas partes -ABQ
    dadosDescida = dados[:metadeDoVetor(dados)].copy()
    dadosSubida = dados[metadeDoVetor(dados):].copy()

    #encontra a row com o valor de campo mais próximo de 0 em ambas metdades da curva -ABQ
    indiceMinDescida = dadosDescida.iloc[:, 0].abs().argmin()
    indiceMinSubida = dadosSubida.iloc[:, 0].abs().argmin()

    #encontra o valor de M/Ms quando o campo é zero -ABQ
    remanenciaDescida = dadosDescida.iloc[indiceMinDescida, 1]
    remanscenciaSubida = dadosSubida.iloc[indiceMinSubida, 1]
    
    #calcula a remancencia deste angulo a partir da media do modulo das remanencias de descida e subida -ABQ
    remanenciaParaAngulo = (abs(remanenciaDescida) + abs(remanscenciaSubida)) / 2
    
    #adiciona o valor de remanencia para este angulo ao vetor de remanencias -ABQ
    remanencias.append([angulo, remanenciaParaAngulo])
    remanenciasDataframe = pd.DataFrame(remanencias)



##### PLOTAGEM

In [ ]:
"""
PLOTANDO A MAGNETIZAÇÃO DE REMANÊNCIA (Mr) PARA CADA ÂNGULO
"""

angulos = remanenciasDataframe.iloc[:, 0].copy() #extrai os angulos -ABQ
magRemanente = remanenciasDataframe.iloc[:, 1].copy() #extrai os valores de remanencia -ABQ

#plota a magnetização de remanência (Mr) para cada ângulo -ABQ
plt.plot(angulos, magRemanente, "--o", color="red", label="Magnetização de Remanência (Mr)")
plt.legend(loc="upper right", frameon=False)

#seta o eixo x de 0 a 180 com ticks de 10 em 10 com espaçamento maior -ABQ
major_ticks = np.arange(0, 181, 20)
plt.xticks(major_ticks)
plt.grid(which="major", axis="x")

minor_ticks = np.arange(0, 181, 10)
plt.xticks(minor_ticks, minor=True)
plt.grid(which="minor", axis="x")

#grade com linhas verticais -ABQ


# Adiciona rótulos e título
plt.xlabel('Ângulo (°)')
plt.ylabel('Mr/Ms (arb. unit)')
plt.title(f"Magnetização de Remanência (Mr) para cada Ângulo {nomeConcatenadoDaAmostraAlvo}")

# Exibe o gráfico
plt.show()

##### Visualização em gráfico polar

In [ ]:
"""
CONSTRUÇÃO DO PLOT POLAR
"""

angulosDup = np.concatenate((angulos, angulos+180)) #duplica os valores de angulos para plotar o gráfico polar -ABQ

magRemanenteDup = np.concatenate((magRemanente, magRemanente[::-1])) #duplica os valores de remanencia para plotar o gráfico polar -ABQ

plt.figure(figsize=(8, 12))
angulosRad = np.deg2rad(angulosDup)
ax = plt.subplot(111, polar=True)
ax.plot(angulosRad, magRemanenteDup, "--o", color="red", label="Mr/Ms")
ax.set_rlabel_position(90)
ax.legend(loc="upper right", frameon=False)
plt.show()

##### Correção de deslocalmento, caso necessário

In [ ]:
"""
DESLOCANDO OS DADOS 10° PARA A DIREITA SE NECESSÁRIO
"""

# Deslocando os dados 10° para a direita
valores_remanencia_deslocados = [(angulo + 10 if angulo + 10 < 360 else angulo + 10 - 360, valor) for angulo, magRemanente in remanenciasDataframe]

"""
DUPLICANDO OS DADOS
"""

# Criando um dicionário para armazenar valores únicos
valores_remanencia_dict = {angulo: valor for angulo, valor in valores_remanencia_deslocados}

# Atualizando novamente os valores de 0° e 360° após duplicação
if 0 not in valores_remanencia_dict:
    valores_remanencia_dict[0] = valores_remanencia_dict.get(350, valores_remanencia_dict.get(10))
if 360 not in valores_remanencia_dict:
    valores_remanencia_dict[360] = valores_remanencia_dict.get(0, valores_remanencia_dict.get(350))

# Duplicando os dados para 0-360 graus, evitando duplicações para ângulo igual
for angulo, valor_remanencia in list(valores_remanencia_deslocados):
    if angulo > 0 and angulo < 180:
        angulo_duplicado = 360 - angulo
        if angulo_duplicado not in valores_remanencia_dict:
            valores_remanencia_dict[angulo_duplicado] = valor_remanencia

# Ordenando os valores duplicados por ângulo
valores_remanencia_duplicados = sorted(valores_remanencia_dict.items())

# Separando ângulos e valores de remanência
angulos = [x[0] for x in valores_remanencia_duplicados]
remanencias = [x[1] for x in valores_remanencia_duplicados]

# Convertendo ângulos para radianos para plotagem polar
angulos_rad = np.radians(angulos)

